In [1]:
# LOAD PACKAGES
# -------------
using JuMP  # Need to say it whenever we use JuMP
using GLPK # Loading the GLPK module for using its solver

# MODEL CONSTRUCTION
# ------------------
sfMipModel = Model(with_optimizer(GLPK.Optimizer)) # No need to chose between LP and MIP 
# (this is chosen automatically) 


A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

In [12]:
# INPUT DATA
# ----------

n = 5 # no. of continuous decision variables
p = 4 # no. of integer decision variables
m = 3 # no. of constraints

# coefficients
A = [0.7511 -0.1357 0.7955 -0.4567 0.1356; -0.6670 -0.3326 0.1657 -0.5519 -0.9367; 1.5894 -0.1302 -0.4313 -0.4875 0.4179]
B = [-0.09520 -0.28056 -1.33978 0.6506; -0.8581 -0.3518 1.2788 1.5114; -0.5925 1.3477 0.1589 0.03495]
c = [0.3468; 0.8687; 0.1200; 0.5024; 0.2884]
d = [0.2017; 0.2712; 0.4997; 0.9238]
f = [0.1716; 0.3610; 0.0705]

1×3 Array{Float64,2}:
 0.1716  0.361  0.0705

In [7]:
# VARIABLES
# ---------

# We work with the two vectors of decision variables
@variable(sfMipModel, x[1:n] >=0) # continuous
@variable(sfMipModel, y[1:p] >= 0, Int) # integer 

ErrorException: An object of name x is already attached to this model. If this is intended, consider using the anonymous construction syntax, e.g., x = @variable(model, [1:N], ...) where the name of the object does not appear inside the macro.

In [19]:
# CONSTRAINTS
# -----------

# We define a vector of constraints
@constraint(sfMipModel, con[i = 1:3], A[i,:]'x + B[i,:]'y == f[i]) 


3-element Array{ConstraintRef{Model,C,Shape} where Shape<:AbstractShape where C,1}:
 con[1] : 0.7511 x[1] - 0.1357 x[2] + 0.7955 x[3] - 0.4567 x[4] + 0.1356 x[5] - 0.0952 y[1] - 0.28056 y[2] - 1.33978 y[3] + 0.6506 y[4] = 0.1716
 con[2] : -0.667 x[1] - 0.3326 x[2] + 0.1657 x[3] - 0.5519 x[4] - 0.9367 x[5] - 0.8581 y[1] - 0.3518 y[2] + 1.2788 y[3] + 1.5114 y[4] = 0.361   
 con[3] : 1.5894 x[1] - 0.1302 x[2] - 0.4313 x[3] - 0.4875 x[4] + 0.4179 x[5] - 0.5925 y[1] + 1.3477 y[2] + 0.1589 y[3] + 0.03495 y[4] = 0.0705 

In [20]:
# OBJECTIVE
# ---------

# Minimize cTx + dTy
@objective(sfMipModel, Min, c'x + d'y) 

# PRINT THE MODEL IN A HUMAN-READABLE FORMAT
# ------------------------------------------
println("The optimization problem to be solved is:")
print(sfMipModel) # Shows the model constructed in a human-readable form


The optimization problem to be solved is:
Min 0.3468 x[1] + 0.8687 x[2] + 0.12 x[3] + 0.5024 x[4] + 0.2884 x[5] + 0.2017 y[1] + 0.2712 y[2] + 0.4997 y[3] + 0.9238 y[4]
Subject to
 0.7511 x[1] - 0.1357 x[2] + 0.7955 x[3] - 0.4567 x[4] + 0.1356 x[5] - 0.0952 y[1] - 0.28056 y[2] - 1.33978 y[3] + 0.6506 y[4] = 0.1716
 -0.667 x[1] - 0.3326 x[2] + 0.1657 x[3] - 0.5519 x[4] - 0.9367 x[5] - 0.8581 y[1] - 0.3518 y[2] + 1.2788 y[3] + 1.5114 y[4] = 0.361
 1.5894 x[1] - 0.1302 x[2] - 0.4313 x[3] - 0.4875 x[4] + 0.4179 x[5] - 0.5925 y[1] + 1.3477 y[2] + 0.1589 y[3] + 0.03495 y[4] = 0.0705
 x[1] ≥ 0.0
 x[2] ≥ 0.0
 x[3] ≥ 0.0
 x[4] ≥ 0.0
 x[5] ≥ 0.0
 y[1] ≥ 0.0
 y[2] ≥ 0.0
 y[3] ≥ 0.0
 y[4] ≥ 0.0
 y[1] integer
 y[2] integer
 y[3] integer
 y[4] integer


In [21]:
# SOLVE IT
# --------

status = optimize!(sfMipModel) # solves the model  

In [22]:
# DISPLAY THE RESULTS
# -------------------

println("Objective value: ", JuMP.objective_value(sfMipModel)) 
println("x = ", JuMP.value.(x))
println("y = ", JuMP.value.(y)) 

Objective value: 1.070277955983598
x = [0.0654907, 0.0, 1.62986, 0.0, 1.22151]
y = [0.0, 0.0, 1.0, 0.0]
